In [1]:
import pandas as pd
import numpy as np

In [2]:
numerical_columns = [
#  'BENE_AGE',
#  'BENE_HI_CVRAGE_TOT_MONS',
#  'BENE_SMI_CVRAGE_TOT_MONS',
 'BENE_HMO_CVRAGE_TOT_MONS',
 'PLAN_CVRG_MOS_NUM',
 'TOTAL_PAY_AMT']
categorical_columns = ['BENE_SEX_IDENT_CD',
 'BENE_RACE_CD',
#  'BENE_ESRD_IND',
 'SP_ALZHDMTA',
 'SP_CHF',
 'SP_CHRNKIDN',
#  'SP_CNCR',
 'SP_COPD',
#  'SP_DEPRESSN',
 'SP_DIABETES',
 'SP_ISCHMCHT',
 'SP_OSTEOPRS',
 'SP_RA_OA',
#  'SP_STRKETIA',
#  'ICD_28521',
#  'ICD_58881',
 'ICD_5990',
#  'ICD_2809',
 'ICD_3051',
 'ICD_311',
 'ICD_5859',
 'ICD_5849',
 'ICD_5856',
 'ICD_27651']


numerical_columns = [
'BENE_AGE',
'BENE_HI_CVRAGE_TOT_MONS',
'BENE_SMI_CVRAGE_TOT_MONS',
'BENE_HMO_CVRAGE_TOT_MONS',
'PLAN_CVRG_MOS_NUM',
'TOTAL_PAY_AMT']
categorical_columns = ['BENE_SEX_IDENT_CD',
'BENE_RACE_CD',
'BENE_ESRD_IND',
'SP_ALZHDMTA',
'SP_CHF',
'SP_CHRNKIDN',
'SP_CNCR',
'SP_COPD',
'SP_DEPRESSN',
'SP_DIABETES',
'SP_ISCHMCHT',
'SP_OSTEOPRS',
'SP_RA_OA',
'SP_STRKETIA',
'ICD_28521',
'ICD_58881',
'ICD_5990',
'ICD_2809',
'ICD_3051',
'ICD_311',
'ICD_5859',
'ICD_5849',
'ICD_5856',
'ICD_27651']

In [3]:
df = pd.read_csv('summ_feed_joined.csv')
#  removed all nulls and infinite values:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
# df = df[numerical_columns+categorical_columns]

# minmaxscaler

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols_to_norm = ['BENE_AGE', 'TOTAL_PAY_AMT','BENE_HI_CVRAGE_TOT_MONS','BENE_SMI_CVRAGE_TOT_MONS','BENE_HMO_CVRAGE_TOT_MONS','PLAN_CVRG_MOS_NUM']

df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])
df
# 
X_variables = df[numerical_columns+categorical_columns]
# X_variables = df
X_encoded = pd.get_dummies(X_variables, columns = ['BENE_SEX_IDENT_CD','BENE_RACE_CD'], dtype = float)




/tmp/ipykernel_8735/2370963542.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]


In [4]:
from sklearn.model_selection import train_test_split
X = X_encoded
y = df['OPIOID_HARMED']

X_train, X_val, y_train, y_val = train_test_split(X,y, random_state = 0, stratify=y)



In [5]:
# from sklearn.neural_network import MLPClassifier


# mlp_gs = MLPClassifier(max_iter=100)
# parameter_space = {
#     'hidden_layer_sizes': [(30,60,30),(40,120,40)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.05],
#     'learning_rate': ['constant','adaptive'],
# }
# from sklearn.model_selection import GridSearchCV
# clf = GridSearchCV(mlp_gs, parameter_space,
#                 n_jobs=-1, 
#                 scoring = 'f1',
#                 cv=5)
# clf.fit(X_train,y_train) # X is train samples and y is the corresponding labels
# print(clf.best_params_)

In [6]:
# {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (30, 60, 30), 'learning_rate': 'adaptive', 'solver': 'adam'}
# nn = MLPClassifier(max_iter=100, activation='tanh', alpha=0.0001, hidden_layer_sizes=(30,60,30), learning_rate='adaptive',solver='adam')
# nn.fit(X_train,y_train)



In [7]:
X_train.shape

(606, 34)

In [8]:
from tensorflow import keras
keras.__version__

'2.3.0-tf'

In [9]:
# from sklearn.metrics import classification_report


# y_predict = nn.predict(y_val)
# classification_report(y_val, y_predict)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam

In [11]:
def get_mlp_model(hidden_layer_one = 256, hidden_layer_two = 128,
            dropout= 0.2, learn_rate = 0.01):
    model = Sequential()
    model.add(Flatten())

    # two stacks of fc -> relu -> dropout
    model.add(Dense(hidden_layer_one, activation = 'relu', input_shape = (756,)))
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layer_two, activation = 'relu'))
    model.add(Dropout(dropout))

    # add softmax layer on top
    model.add(Dense(10, activation = 'softmax'))

    # compile model
    model.compile(
        optimizer= Adam(learning_rate = learn_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']

    )

    return model

In [12]:
model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[34, 1]),
        keras.layers.Dense(40, activation="relu"),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 34)                0         
_________________________________________________________________
dense (Dense)                (None, 40)                1400      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________


2022-10-11 17:58:51.633867: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/python/3.8.2/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/ipp/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/mkl/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/tbb/lib/intel64/gcc4.7:/N/soft/rhel7/intel/19.5/debugger_2019/iga/lib:/N/soft/rhel7/intel/19.5/debugger_2019/libipt/intel64/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/daal/lib/intel64_lin:/N/soft/rhel7/gcc/6.3.0/lib64:/N/soft/rhel7/gcc/6.3.0/lib
2022-10-11 17:58:51.633930: E tensorflow/strea

In [13]:
model.compile(loss="binary_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])

model.fit(x=X_train, y= y_train, validation_data=(X_val, y_val), batch_size =8, epochs= 20)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/N/u/hanjos/Carbonate/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
  File "/tmp/ipykernel_8735/2641924897.py", line 5, in <module>
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 66, in _method_wrapper
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 802, in fit
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 1100, in __init__
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 364, in __init__
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 390, in slice_inputs
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", lin

In [14]:
import sys
print("%x" % sys.maxsize, sys.maxsize > 2**32)

7fffffffffffffff True


In [15]:
import tensorflow as tf 
# tf.Session()
# tf.ConfigProto()
# one
# print('hi')
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('CPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/N/u/hanjos/Carbonate/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
  File "/tmp/ipykernel_8735/3379309374.py", line 8, in <module>
  File "/N/soft/rhel7/python/3.8.2/lib/python3.8/site-packages/tensorflow/python/client/device_lib.py", line 43, in list_local_devices
RuntimeError: Resource temporarily unavailable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/N/u/hanjos/Carbonate/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
  File "/N/u/hanjos/Carbonate/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
  File "/N/u/hanjos/Carbonate/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1006, in structured_traceback
  File "/N/u/hanjos/Carbonate/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 859, in structu

In [16]:
# model.fit(X_train, y_train)

In [17]:
# import tensorflow as tf
# tf.random.set_seed(42)


In [18]:
# model = get_mlp_model()
# H = model.fit(x=X_train, y= y_train, validation_data=(X_val, y_val), batch_size =8, epochs= 20)

# accuracy = model.evaluate(X_val, y_val)
# accuracy